## Two-way ANOVA test
Enables to test the effect of two factors at the same time a continuous variable. 

Restriction: the number of observations in each cell has to be equal

In [ ]:
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.factorplots import interaction_plot
import statsmodels.api as sm
import pandas as pd
from scipy import stats
import json
import seaborn
%matplotlib inline

In [ ]:
seek_f = open('JSON_INPUT')

In [ ]:
# check if input is a number by casting it to a float
# https://stackoverflow.com/questions/354038/how-do-i-check-if-a-string-is-a-number-float
def is_number(item):
    try:
        float(item)
        return True
    except ValueError:
        return False

# Returns True if item is either a number or a string (if it is castable to either float or string)
def is_number_or_string(item):
    try:
        float(item)
        return True
    except ValueError:
        str(item)
        return True
    except:
        print "Error: cannot cast value neither to float nor a string - ", item
    return False

# re-structure to get rid of spreadsheet junk and convert the values 
def restructure_as_ordered_dict(j):
    retval = OrderedDict()
    for (k,v) in j.items():
        #Get the heading of each column 
        first = v["values"][0].encode('utf-8');

        #Create a new ordered dict with keys being the column headings, and the values being the column data
        effective_key = "";
        if(not is_number(first)):
            retval[first] = v["values"][1:] #the column data starts after the heading (row 1+)
            effective_key = first 
        else:
            retval[k] = v;
            effective_key = k;
        
        #filter values by either string or numbers, and perform the actual conversion (map)
        retval[effective_key] = list(map(lambda x: float(x) if is_number(x) else str(x),  
                                         filter(is_number_or_string, retval[effective_key]) ) )

    del retval[''] #remove index column
    return retval

# now read the file, parse content & create the proper DataFrame
seek_f_content = seek_f.read();
seek_f_json = json.loads(seek_f_content, object_pairs_hook=OrderedDict);

marked = seek_f_json["marked"];

data_sep = restructure_as_ordered_dict(marked)
data = pd.DataFrame(data_sep, columns=data_sep.keys())

In [ ]:
# Prepare the needed info for 2-way ANOVA

# return a single dataframe column (i) as a Series object using squeeze()
def get_df_col(df,i):
    return df.iloc[:,i:i+1].squeeze()

# indexes in the Dataframe
ci = 0    #continuous variable (dependent)
f1_i = 1  #factor 1 column index (indepedent)
f2_i = 2  #factor 2 column index (indepedent)

# easy access to each column, prepared in a Series object 
ci_col = get_df_col(data, ci)   
f1_col = get_df_col(data, f1_i) 
f2_col = get_df_col(data, f2_i) 

### Exploratory histogram 

In [ ]:
lab = "N = %d\nmean = %.1f\nstd = %.1f" %(len(ci_col), 
                                          np.mean(ci_col), 
                                          np.std(ci_col))
labels = [lab]
n, bins, patches = plt.hist(ci_col, alpha=0.5, label=labels)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### Interaction plot

In [ ]:
fig = interaction_plot(f2_col, f1_col, ci_col,
             colors=['red','blue'], markers=['D','^'], ms=10)

### Compute ANOVA 

In [ ]:
# calculate Degrees of Freedom (dof)
N = len(ci_col)
dof_a = len(f1_col.unique()) - 1     #factor 1
dof_b = len(f2_col.unique()) - 1     #factor 2Fa
dof_axb = dof_a*dof_b 
dof_w = N - (len(f1_col.unique())*len(f2_col.unique()))

#calculate Sum of Squares for each factor (indepedent variables)
grand_mean = ci_col.mean()
ssq_a = sum([(data[f1_col ==l].len.mean()-grand_mean)**2 for l in f1_col])
ssq_b = sum([(data[f2_col ==l].len.mean()-grand_mean)**2 for l in f2_col])

#total Sum of Squares
ssq_t = sum((ci_col - grand_mean)**2)

#means in each treatment group, i.e each combination of the factors
f1_dict = {f1_lvl: data[f1_col == f1_lvl] for f1_lvl in f1_col.unique()}    
f1_x_f2_dict = { k: [get_df_col(v[get_df_col(v,f2_i) == f2_c] ,ci).mean() for f2_c in get_df_col(v,f2_i)] for k, v in f1_dict.iteritems()}    

#get Sum of Squares Within (residuals)
ssq_w = 0
for k in f1_dict.keys():
    ssq_w +=sum((get_df_col(f1_dict[k],ci)-f1_x_f2_dict[k])**2)

#Sum of Squares of the interaction between the factors 
ssq_axb = ssq_t-ssq_a-ssq_b-ssq_w

#Mean Squares
ms_a = ssq_a/dof_a
ms_b = ssq_b/dof_b
ms_axb = ssq_axb/dof_axb
ms_w = ssq_w/dof_w

#F-ratio: the F-statistic is the mean square for each effect and the interaction divided by the mean square within
f_a = ms_a/ms_w
f_b = ms_b/ms_w
f_axb = ms_axb/ms_w

#P-values
#scipy.stats method f.sf checks if the obtained F-ratios are above the critical value. 
#Null Hypotheses 1/2:  H0: no difference in means among the groups in factor 1/2
p_a = stats.f.sf(f_a, dof_a, dof_w)
p_b = stats.f.sf(f_b, dof_b, dof_w)
#Null Hypothesis 3:  H0: no interaction 
p_axb = stats.f.sf(f_axb, dof_axb, dof_w)

In [ ]:
results = {'sum_sq':[ssq_a, ssq_b, ssq_axb, ssq_w],
          'dof':[dof_a, dof_b, dof_axb, dof_w],
          'MS': [ms_a, ms_b, ms_axb, ms_w],
          'F':[f_a, f_b, f_axb, 'NaN'],
           'PR(>F)':[p_a, p_b, p_axb, 'NaN']}
columns=['sum_sq', 'dof', 'MS', 'F', 'PR(>F)']
 
aov_table1 = pd.DataFrame(results, columns=columns,
                         index=[data.columns.values[f1_i], data.columns.values[f2_i], 
                         data.columns.values[f1_i]+":"+data.columns.values[f2_i], 'Residual'])


In [ ]:
aov_table1

### QQplot of the residuals 

In [ ]:
# Get the residuals into a QQplot
df_resid = pd.DataFrame()
for k in f1_dict.keys():
    #compute the residuals without sum of squares, turn to a dataframe
    st =  ( (get_df_col(f1_dict[k],ci)-f1_x_f2_dict[k]) ).to_frame()   
    df_resid = pd.concat([df_resid, st])

sm.qqplot(df_resid[data.columns.values[ci]], line='s')
plt.show()

### Pairplot 
Hue based on factor 1

In [ ]:
# Pairplot 
seaborn.pairplot(data, vars=[data.columns.values[ci], data.columns.values[f2_i]], kind='reg', hue=data.columns.values[f1_i])